loading libraries

In [85]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

dataset

In [86]:
df_dataset_original = pd.read_csv('datasets/olist_orders_dataset.csv')
df_order = df_dataset_original.copy()

initial visualization

In [87]:
df_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


 items dataframe

In [88]:
df_items_original = pd.read_csv('datasets/olist_order_items_dataset.csv')
df_items = df_items_original.copy()

In [89]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


null values

In [90]:
df_order.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [91]:
df_order = df_order.dropna(subset=["order_delivered_customer_date", "order_delivered_carrier_date", "order_approved_at"])

In [92]:
df_order.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

combine dataframes orders info and orders

In [93]:
df_combined = df_order.merge(df_items, on='order_id',how='inner')

convert dates to datetime format

In [94]:
df_combined['order_purchase_timestamp'] = pd.to_datetime(df_combined['order_purchase_timestamp'])
df_combined['order_approved_at'] = pd.to_datetime(df_combined['order_approved_at'])
df_combined['order_delivered_carrier_date'] = pd.to_datetime(df_combined['order_delivered_carrier_date'])
df_combined['order_delivered_customer_date'] = pd.to_datetime(df_combined['order_delivered_customer_date'])
df_combined['order_estimated_delivery_date'] = pd.to_datetime(df_combined['order_estimated_delivery_date'])


calculate the real time delivery

In [95]:
df_combined['delivery_time'] = (df_combined['order_delivered_customer_date'] - df_combined['order_purchase_timestamp']).dt.days

splitting the datasets

In [96]:
X = df_combined[['price', 'freight_value']]
y = df_combined['delivery_time'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train the model

In [97]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

evaluate the model

In [98]:
y_pred = model.predict(X_test)
print("Mean Absolute Error: ", mean_absolute_error(y_test, y_pred))

Mean Absolute Error:  6.08720118456212
